#### Imports defined

In [1]:
import pandas as pd

#### Reading the csv file and storing it into a DataFrame.

In [2]:
df = pd.read_csv('data/esb.csv')

In [3]:
df.head()

,Unnamed: 0,Total ESB Connections - Monthly by area 2006-2013,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104
0,NaN,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,County Councils,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sept,...,Apr,May,Jun,Jul,Aug,Sept,Oct,Nov,Dec,Total
2,Carlow,70,81,113,56,97,125,158,141,96,...,10,8,26,21,5,28,14,14,16,169
3,Cavan,149,208,197,261,201,248,308,199,237,...,17,13,10,15,16,11,11,14,21,173
4,Clare,171,251,264,200,242,324,195,164,155,...,21,22,15,19,14,13,19,26,20,215


#### Extracting specific year values from the DataFrame.

In [4]:
year_values = [ i for i in df.loc[0] if not isinstance(i,float)]

#### First row in the DataFrame in filled up with NaN values. Clearing them up.

In [5]:
df = df.drop(df.index[0])

In [6]:
df.head()

,Unnamed: 0,Total ESB Connections - Monthly by area 2006-2013,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104
1,County Councils,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sept,...,Apr,May,Jun,Jul,Aug,Sept,Oct,Nov,Dec,Total
2,Carlow,70,81,113,56,97,125,158,141,96,...,10,8,26,21,5,28,14,14,16,169
3,Cavan,149,208,197,261,201,248,308,199,237,...,17,13,10,15,16,11,11,14,21,173
4,Clare,171,251,264,200,242,324,195,164,155,...,21,22,15,19,14,13,19,26,20,215
5,Cork,588,615,676,554,702,602,702,592,630,...,69,83,67,86,77,82,81,90,67,863


#### Based on the index position of the rows in the DataFrame, slicing it to separate the records for various counties.

In [7]:
df_county = df.loc[2:30]

#### Similarly separating the records for the cities.

In [8]:
df_city = df.loc[32:36]

#### Function to format a DataFrame. This function aims to clean up a DataFrame. Below are the steps of operation.
1. It begins by changing the index of the DataFrame, replacing it with the values found in column <b>Umnamed : 0</b>
2. The numeric range for the columns in the DataFrame is calulated. It is for iterating over the DataFrame.
3. The columns are in a multiple of 13 (it being a prime number also helps) for each of the years. Every group of 12 columns (Jan-Dec) is made into a separetae DataFrame which are the ESB connection values for every month in each of the years.
4. Every sliced DataFrame is them inserted with two new columns namely <b>Council Type</b> and <b>Year</b>. It records the type of the Council viz. County and City along with the year for which the values correspond.
5. All the smaller DataFrames are then concatenated to form a single DataFrame.
6. Some columns had dirty data, numbers preceeded with <b>#</b> symbol. This function also removes this anomaly.

#### The output of this function is a DataFrame with less number of columns and increaseed numbe of rows in comparision with the input DataFrame.

<b>Note</b>: As part of this formatting, column <b>Totals</b> and the rows <b>TOTALS</b> and <b>Conversions</b> are removed.

In [9]:
def format_dataframe(df,Type):
    
    col_list = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec']
    
    new_df = pd.DataFrame()
    col_indexes = []
    counter = 0
    
    df = df.set_index('Unnamed: 0')
    
    df.columns = range(1, (df.shape[1] + 1) )
    
    for i in df.columns:
        if not i % 13 == 0 :
            col_indexes.append(i)

        else:
            start = col_indexes[0]
            end = col_indexes[(len(col_indexes) - 1)]

            col_indexes.clear()
            sliced_df = df.loc[:,start:end]

            if counter < len(year_values):
                
                sliced_df.columns = col_list
                sliced_df.insert(0,'Council Type',Type)
                sliced_df.insert(1,'Year',year_values[counter])

                new_df = pd.concat([new_df, sliced_df], ignore_index=False)
                counter+= 1
                
    new_df.reset_index(level=0, inplace=True)        
    new_df.rename({'Unnamed: 0':'Council Name'},axis='columns',inplace=True)
    
    for i in col_list:
        new_df[i] = new_df[i].str.replace('#','')
    
    new_df[col_list] = new_df[col_list].apply(pd.to_numeric, errors = 'coerce')
                
    return new_df

#### Function to tidy a DataFrame. This function shrinks the columns of the  DataFrame and records each of the separate observations for months (Jan-Dec) under a single column. 


In [10]:
def tidy_up(passed_df):
    
    tidy_df = pd.DataFrame(columns = ['Council Name','Council Type','Year','Month','ESB Connection'])
    months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec']

    for i in passed_df.index:
        constants = list(passed_df.iloc[i,0:3])
        monthly_values = list(passed_df.iloc[i,3:])

        for j,k in enumerate(monthly_values):
            row = {
                   'Council Name':constants[0],
                   'Council Type':constants[1],
                   'Year':constants[2],
                   'Month':months[j],
                   'ESB Connection':k
                  }
            df = pd.DataFrame(row,index = [i])
            tidy_df = pd.concat([tidy_df,df], ignore_index=True)
            
    return tidy_df

#### Formatting the DataFrame that has the records of various cities.

In [11]:
df_city_formatted = format_dataframe(df_city,'City Council')

In [12]:
df_city_formatted.head()

,Council Name,Council Type,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sept,Oct,Nov,Dec
0,Cork,City Council,2006,39,22,93,31,162,110,57,141,96,103,108,103
1,Dublin,City Council,2006,757,618,560,225,496,717,685,783,847,690,755,613
2,Galway,City Council,2006,68,107,57,72,199,75,48,55,229,137,142,86
3,Limerick,City Council,2006,19,10,80,32,17,22,17,11,40,12,50,79
4,Waterford,City Council,2006,69,33,91,34,63,68,116,82,42,82,85,63


#### Formatting the DataFrame that has the records of various counties.

In [13]:
df_county_formatted = format_dataframe(df_county, 'County Council')

In [14]:
df_county_formatted.head()

,Council Name,Council Type,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sept,Oct,Nov,Dec
0,Carlow,County Council,2006,70,81,113,56,97,125,158,141,96,142,182,60
1,Cavan,County Council,2006,149,208,197,261,201,248,308,199,237,203,259,219
2,Clare,County Council,2006,171,251,264,200,242,324,195,164,155,252,296,213
3,Cork,County Council,2006,588,615,676,554,702,602,702,592,630,810,900,705
4,Donegal,County Council,2006,285,450,342,217,365,439,350,276,328,475,503,357


#### Merging the above formatted DataFrames.

In [15]:
merged_df = pd.concat([df_county_formatted,df_city_formatted], ignore_index=True)

merged_df.sort_values(['Year', 'Council Type'],inplace=True)

merged_df.reset_index(drop=True, inplace=True)

#### Tidying the above merged DataFrame.

In [16]:
tidy_df = tidy_up(merged_df)

In [17]:
tidy_df.head()

,Council Name,Council Type,Year,Month,ESB Connection
0,Cork,City Council,2006,Jan,39
1,Cork,City Council,2006,Feb,22
2,Cork,City Council,2006,Mar,93
3,Cork,City Council,2006,Apr,31
4,Cork,City Council,2006,May,162


#### Saving the tidied DataFrame as a csv file.

In [18]:
tidy_df.to_csv('data/esb_tidy.csv',index = False)
tidy_df.to_csv('C00246376_Python_Assignment_3_Part_3/data/esb_tidy.csv',index = False)